In [ ]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from src.data_handler import DataHandler
from src.analysis_logic import FitMethods
from src.helper_functions import format_exponent_as_str

plt.style.use('seaborn')
plt.rcParams['image.cmap'] = sns.color_palette("mako", as_cmap=True)

In [ ]:
tip_3B4_uhv = DataHandler(measurement_folder="20220804_FR0612-F2-3B6_uhv_new_objective")

In [ ]:
tip_3B4_uhv_measurements = tip_3B4_uhv.load_measurement_list_into_dataclass_list([
        r"Confocal\20220809-1207-42_higher_laser_power_confocal_xy_image_1_fig",
        r"Confocal\20220809-1233-48_higher_laser_power_confocal_xy_image_1_fig"
    ]
)

In [ ]:
confocal = tip_3B4_uhv_measurements[0]

fig, ax = plt.subplots()
params = confocal.params
extent = np.array([params['X image min (m)'], params['X image max (m)'], params['Y image min'], params['Y image max']]) * 1e3
img = ax.imshow(confocal.data / 1e3, origin="lower", extent=extent, vmax=120)
cbar = plt.colorbar(img)

ax.set_xlabel("X (μm)")
ax.set_ylabel("Y (μm)")

ax.grid(False)

confocal.filepath

confocal1 = confocal

In [ ]:
confocal = tip_3B4_uhv_measurements[1]

fig, ax = plt.subplots(figsize=(6, 4))
params = confocal.params
extent = np.array([params['X image min (m)'], params['X image max (m)'], params['Y image min'], params['Y image max']]) * 1e3
img = ax.imshow(confocal.data / 1e3, origin="lower", extent=extent, vmax=30, interpolation="hanning")
cbar = plt.colorbar(img)
cbar.set_label("Fluoresence (kc/s)")

ax.set_xlabel("X (μm)")
ax.set_ylabel("Y (μm)")

ax.grid(False)

ax.set_xlim([1, 9])
ax.set_ylim([2, 9])

tip_3B4_uhv.save_figures(fig, filename=confocal.filename)

## Confocal as a function of pressure

In [ ]:
tip_3B4_uhv_confocal_list = tip_3B4_uhv.load_measurements_into_dataclass_list(measurement_str="Confocal")

In [ ]:
for confocal in tip_3B4_uhv_confocal_list:
    if "mbar" in confocal.filename:
        print(confocal)

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(20, 15))

idx = 0
for confocal in tip_3B4_uhv_confocal_list:
    if "mbar" in confocal.filename:
        params = confocal.params
        extent = np.array([params['X image min (m)'], params['X image max (m)'], params['Y image min'], params['Y image max']]) * 1e3
        img = ax[idx].imshow(confocal.data[:, 3].reshape(120, 120) / 1e3, origin="lower", extent=extent, vmax=30, vmin=1)

        pressure = confocal.get_param_from_filename(unit="mbar")
        ax[idx].set_title(f"{format_exponent_as_str(pressure)} mbar\n{confocal.timestamp.strftime('%d-%m-%y %H:%M')}")
        ax[idx].grid(False)
        
        idx += 1

cbar = fig.colorbar(img, ax=ax.ravel().tolist(), shrink=0.25)
tip_3B4_uhv.save_figures(fig, filename="confocal_at_different_pressures", bbox_inches="tight")

# AFM

In [ ]:
freq_sweep_measurements = tip_3B4_uhv.load_measurements_into_dataclass_list(measurement_str="frq-sweep")

## Akiyama

In [ ]:
akiyama = freq_sweep_measurements[0]

fig, (ax, ax1) = plt.subplots(nrows=2, sharex=True)

ax.plot(akiyama.params["f_res (Hz)"] / 1e3 + akiyama.data["Frequency Shift (Hz)"] / 1e3, akiyama.data["Amplitude (m)"] * 1e9)
ax1.plot(akiyama.params["f_res (Hz)"] / 1e3 + akiyama.data["Frequency Shift (Hz)"] / 1e3, akiyama.data["Phase (deg)"])

ax.set_ylabel("Amplitude (nm)")
ax1.set_ylabel("Phase (deg)")

ax1.set_xlabel("Frequency Shift (kHz)")

ax.set_title(f"Akiyama Q={akiyama.params['Q']}")

## Tip 3B4

In [ ]:
diamond_3B4 = freq_sweep_measurements[8]

fig, (ax, ax1) = plt.subplots(nrows=2, sharex=True)

ax.plot(diamond_3B4.params['Center Frequency (Hz)'] / 1e3 + diamond_3B4.data["Frequency Shift (Hz)"] / 1e3, diamond_3B4.data["Amplitude (m)"] * 1e9)
ax1.plot(diamond_3B4.params['Center Frequency (Hz)'] / 1e3 + diamond_3B4.data["Frequency Shift (Hz)"] / 1e3, diamond_3B4.data["Phase (deg)"])

ax.set_ylabel("Amplitude (nm)")
ax1.set_ylabel("Phase (deg)")

ax1.set_xlabel("Frequency Shift (kHz)")

ax.axvline(diamond_3B4.params["f_res (Hz)"] / 1e3, linestyle="--", color="C1")
ax1.axvline(diamond_3B4.params["f_res (Hz)"] / 1e3, linestyle="--", color="C1")

ax.set_title(f"Diamond Tip Q={diamond_3B4.params['Q']}")

## Tip 3G6

In [ ]:
diamond_3G6 = freq_sweep_measurements[24]

fig, (ax, ax1) = plt.subplots(nrows=2, sharex=True)

ax.plot(diamond_3G6.params['Center Frequency (Hz)'] / 1e3 + diamond_3G6.data["Frequency Shift (Hz)"] / 1e3, diamond_3G6.data["Amplitude (m)"] * 1e9)
ax1.plot(diamond_3G6.params['Center Frequency (Hz)'] / 1e3 + diamond_3G6.data["Frequency Shift (Hz)"] / 1e3, diamond_3G6.data["Phase (deg)"])

ax.set_ylabel("Amplitude (nm)")
ax1.set_ylabel("Phase (deg)")

ax1.set_xlabel("Frequency Shift (kHz)")

ax.axvline(diamond_3G6.params["f_res (Hz)"] / 1e3, linestyle="--", color="C1")
ax1.axvline(diamond_3G6.params["f_res (Hz)"] / 1e3, linestyle="--", color="C1")

ax.set_title(f"Diamond Tip Q={diamond_3G6.params['Q']}")

## Comparison

In [ ]:
fig, (ax, ax1) = plt.subplots(nrows=2, sharex=True)

ax.plot(diamond_3B4.data["Frequency Shift (Hz)"] / 1e3, diamond_3B4.data["Amplitude (m)"] * 1e9, label=f"Tip 3B4, Q={round(diamond_3B4.params['Q'])}")
ax1.plot(diamond_3B4.data["Frequency Shift (Hz)"] / 1e3, diamond_3B4.data["Phase (deg)"])

ax.plot(diamond_3G6.data["Frequency Shift (Hz)"] / 1e3, diamond_3G6.data["Amplitude (m)"] * 1e9, label=f"Tip 3G6, Q={round(diamond_3G6.params['Q'])}")
ax1.plot(diamond_3G6.data["Frequency Shift (Hz)"] / 1e3, diamond_3G6.data["Phase (deg)"])

ax.plot(akiyama.data["Frequency Shift (Hz)"] / 1e3, akiyama.data["Amplitude (m)"] * 1e9, label=f"Akiyama, Q={round(akiyama.params['Q'])}")
ax1.plot(akiyama.data["Frequency Shift (Hz)"] / 1e3, akiyama.data["Phase (deg)"])

ax.set_ylabel("Amplitude (nm)")
# ax.set_yscale("log")

ax1.set_ylabel("Phase (deg)")

ax1.set_xlabel("Frequency Shift (kHz)")

ax.legend()

# tip_3B4_uhv.save_figures(fig, filename="comparision_akiyama_3B4_3G6")

## Variation with voltage

In [ ]:
fig, (ax, ax1) = plt.subplots(nrows=2, sharex=True)

differences = []
voltages = []

for freq_sweep in freq_sweep_measurements[12:21]:
    freq = freq_sweep.data["Frequency Shift (Hz)"] / 1e3
    amp = freq_sweep.data["Amplitude (m)"] / np.max(freq_sweep.data["Amplitude (m)"])
    phase = freq_sweep.data["Phase (deg)"]
    
    ax.plot(freq,  amp)
    ax1.plot(freq, phase, label=f"V={freq_sweep.params['Excitation (V)']} V, Q={round(freq_sweep.params['Q'])}")
    
    differences.append(np.max(amp) - np.min(amp))
    voltages.append(freq_sweep.params['Excitation (V)'])

# ax.set_yscale("log")
ax1.legend()

# tip_3B4_uhv.save_figures(fig, filename="variation_with_voltage")

In [ ]:
fig, ax = plt.subplots()

ax.plot(voltages, differences, "o")
ax.set_xscale("log")

ax.set_xlabel("Voltage (V)")
ax.set_ylabel("Amplitude contrast (a.u.)")

ax.axvspan(0.025, 0.1, alpha=0.5, color="C1")

# Tip 2K14